# GTEx model building with CLAMP and PLIER

💡 **Environment:** `clamp-analyses`  

This notebook builds latent variable models from GTEx v8 RNA‑seq TPM data using CLAMP and PLIER. It automates downloading and preprocessing the GTEx matrix, creates a Filebacked Big Matrix (FBM), computes an SVD to estimate the model dimension, prepares pathway priors, runs CLAMP (base + full) and PLIER, and saves model outputs (B, Z, summaries) and intermediate files. Configuration and paths are controlled via `config.R`.

# CLAMP

## Load libraries

In [1]:
# Create a timestamp to track the start of the analysis
start_time <- Sys.time()
cat("GTEx CLAMP and PLIER analysis started at:", format(start_time), "\n")

GTEx CLAMP and PLIER analysis started at: 2025-11-25 13:01:35 


In [2]:
if (!requireNamespace("PLIER", quietly = TRUE)) {
    devtools::install_github("wgmao/PLIER")
}

library(bigstatsr)
library(data.table)
library(dplyr)
library(rsvd)
library(glmnet)
library(Matrix)
library(knitr)
library(here)
library(PLIER)
library(CLAMP)

source(here("config.R"))

set.seed(config$GTEx$RANDOM_SVD_SEED)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: Matrix

Loaded glmnet 4.1-10

here() starts at /home/msubirana/Documents/pivlab/clamp-analyses

Loading required package: RColorBrewer

Loading required package: gplots


Attaching package: ‘gplots’


The following object is masked from ‘package:stats’:

    lowess


Loading required package: pheatmap

Loading required package: qvalue


Attaching package: ‘CLAMP’


The following object is masked from ‘package:PLIER’:

    num.pc




## Output directory

In [3]:
output_data_dir <- config$GTEx$DATASET_FOLDER
dir.create(output_data_dir, showWarnings = FALSE, recursive = TRUE)

# Settings

In [4]:
block_size <- config$GENERAL$CHUNK_SIZE
N_CORES    <- config$GTEx$N_CORES

## Download GTEx 

In [5]:
url <- config$GTEx$URL
dest_dir <-  config$GTEx$DATASET_FOLDER
dest_gz  <- file.path(dest_dir, basename(url))

if (!file.exists(dest_gz)) {
  dir.create(dest_dir, recursive = TRUE, showWarnings = FALSE)
  download.file(url, dest_gz, mode = "wb")
  message("Downloaded to: ", dest_gz)
} else {
  message("File already exists, skipping download.")
}

File already exists, skipping download.



## Preprocess GTEx data

In [6]:
exprs_path  <- file.path(config$GTEx$DATASET_FOLDER, 'GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct.gz')
output_file <- config$GTEx$DATASET_FILE

if (!file.exists(output_file)) {
  dir.create(dirname(output_file), recursive = TRUE, showWarnings = FALSE)
  exprs_data <- read.table(exprs_path, header = TRUE, sep = "\t", skip = 2, check.names = FALSE)
  saveRDS(exprs_data, config$GTEx$DATASET_FILE)
  message("File successfully written to: ", config$GTEx$DATASET_FILE)
} else {
  message("Output file already exists. Skipping.")
}

# Aggregate in-place by 'description'
gtex <- readRDS(here(config$GTEx$DATASET_FILE))
gtex <- as.data.table(gtex)
aggregated_gtex <- gtex[, lapply(.SD, sum), by = Description, .SDcols = is.numeric]

genes <- aggregated_gtex$Description
samples <- colnames(aggregated_gtex[, -1])
data_mat <- as.matrix(aggregated_gtex[, -1])

Output file already exists. Skipping.



In [ ]:
fbm_files <- c(
  file.path(output_data_dir, "FBMgtex.bk"),
  file.path(output_data_dir, "FBMgtex_preproc.bk"),
  file.path(output_data_dir, "FBMgtex_preproc_filtered.bk"),
  file.path(output_data_dir, "FBMgtex_preproc_filtered.rds")
)

for (f in fbm_files) {
  if (file.exists(f)) {
    res <- unlink(f)
    if (identical(res, 0L)) {
      message("✅ Removed existing FBM file: ", f)
    } else {
      warning("⚠️ Failed to remove: ", f)
    }
  } else {
    message("ℹ️ Not found, skipping: ", f)
  }
}

✅ Removed existing FBM file: /home/msubirana/Documents/pivlab/clamp-analyses/output/gtex/FBMgtex.bk

✅ Removed existing FBM file: /home/msubirana/Documents/pivlab/clamp-analyses/output/gtex/FBMgtex.bk_preproc.bk

✅ Removed existing FBM file: /home/msubirana/Documents/pivlab/clamp-analyses/output/gtex/FBMgtex.bk_preproc_filtered.bk

ℹ️ Not found, skipping: /home/msubirana/Documents/pivlab/clamp-analyses/output/gtex/FBMgtex.bk_preproc_filtered.rds



In [9]:
# Create the FBM
fbm_file <- file.path(output_data_dir, "FBMgtex")
gtexFBM <- FBM(nrow = nrow(data_mat), ncol = ncol(data_mat), backingfile = fbm_file, create_bk = T)

# Populate it with data
n_blocks <- ceiling(nrow(aggregated_gtex) / block_size)

for (i in 1:n_blocks) {
  start_row <- (i-1) * block_size + 1
  end_row <- min(i * block_size, nrow(data_mat))
  
  gtexFBM[start_row:end_row, ] <- as.matrix(data_mat[start_row:end_row, ])
}

# Preprocess and z‑score FBM

prep_gtex <- preprocessCLAMPFBM(
  fbm        = gtexFBM,
  mean_cutoff= config$GTEx$GENES_MEAN_CUTOFF,
  var_cutoff = config$GTEx$GENES_VAR_CUTOFF,
  ncores=N_CORES
)

gtex_fbm_filt <- prep_gtex$fbm_filtered
gtex_rowStats <- prep_gtex$rowStats

Applying log2 transformation

No NA values found



Check overlap genes with GTEx v8 RNA-seq data and TWAS gene list

In [10]:
twas_genes <- read.csv(here('output/creating_twas_gwas_list/gene_list_union_mashr_and_elastic_net_with_phi.csv'))
head(twas_genes)

,Ensembl_ID,Gene_Symbol,Ensembl_ID_w_version,en_Adipose_Subcutaneous,en_Adipose_Visceral_Omentum,en_Adrenal_Gland,en_Artery_Aorta,en_Artery_Coronary,en_Artery_Tibial,en_Brain_Amygdala,⋯,mashr_Skin_Not_Sun_Exposed_Suprapubic,mashr_Skin_Sun_Exposed_Lower_leg,mashr_Small_Intestine_Terminal_Ileum,mashr_Spleen,mashr_Stomach,mashr_Testis,mashr_Thyroid,mashr_Uterus,mashr_Vagina,mashr_Whole_Blood
,<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,ENSG00000238009,RP11-34P13.7,ENSG00000238009.6,0,0,0,0,0,0,0,⋯,0,0,0,1,0,1,0,0,0,1
2,ENSG00000228463,AP006222.2,ENSG00000228463.9,0,1,1,0,0,0,0,⋯,1,1,0,0,1,1,1,1,1,0
3,ENSG00000237094,RP4-669L17.10,ENSG00000237094.11,0,0,0,0,0,0,0,⋯,1,0,0,0,0,0,1,0,0,0
4,ENSG00000230021,RP5-857K21.4,ENSG00000230021.8,0,0,0,0,0,0,0,⋯,0,0,0,0,0,1,1,0,0,0
5,ENSG00000237491,RP11-206L10.9,ENSG00000237491.8,1,1,1,1,1,1,1,⋯,1,1,1,1,1,1,1,1,1,1
6,ENSG00000177757,FAM87B,ENSG00000177757.2,1,1,1,1,1,1,0,⋯,1,1,1,1,1,1,1,1,1,1


In [11]:
overlap_genes <- intersect(genes[prep_gtex$kept_rows], twas_genes$Gene_Symbol)

percent_overlap <- (length(overlap_genes) / length(twas_genes$Gene_Symbol)) * 100

cat("Number of overlapping genes:", length(overlap_genes), "\n")
cat("Percentage overlap (relative to TWAS genes):", round(percent_overlap, 2), "%\n")

Number of overlapping genes: 16488 


Percentage overlap (relative to TWAS genes): 72.24 %


In [12]:
zscoreCLAMPFBM(gtex_fbm_filt, gtex_rowStats, ncores=N_CORES)
gtex_genes <- genes[prep_gtex$kept_rows]

Applying Z-score transformation



In [13]:
saveRDS(samples, file = file.path(output_data_dir, "gtex_samples.rds"))

In [14]:
saveRDS(gtex_genes, file = file.path(output_data_dir, "gtex_genes.rds"))

In [15]:
saveRDS(gtex_fbm_filt, file = file.path(output_data_dir, "gtex_fbm_filt.rds"))

In [16]:
df_gtex_fbm_filt <- as.data.frame(as.matrix(gtex_fbm_filt[]))

In [17]:
colnames(df_gtex_fbm_filt) <- samples

In [18]:
rownames(df_gtex_fbm_filt) <- gtex_genes

In [19]:
head(df_gtex_fbm_filt)

,GTEX-1117F-0226-SM-5GZZ7,GTEX-1117F-0426-SM-5EGHI,GTEX-1117F-0526-SM-5EGHJ,GTEX-1117F-0626-SM-5N9CS,GTEX-1117F-0726-SM-5GIEN,GTEX-1117F-1326-SM-5EGHH,GTEX-1117F-2426-SM-5EGGH,GTEX-1117F-2526-SM-5GZY6,GTEX-1117F-2826-SM-5GZXL,GTEX-1117F-2926-SM-5GZYI,⋯,GTEX-ZZPU-1126-SM-5N9CW,GTEX-ZZPU-1226-SM-5N9CK,GTEX-ZZPU-1326-SM-5GZWS,GTEX-ZZPU-1426-SM-5GZZ6,GTEX-ZZPU-1826-SM-5E43L,GTEX-ZZPU-2126-SM-5EGIU,GTEX-ZZPU-2226-SM-5EGIV,GTEX-ZZPU-2426-SM-5E44I,GTEX-ZZPU-2626-SM-5E45Y,GTEX-ZZPU-2726-SM-5NQ8O
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
WASH7P,1.44614308,0.1466590,1.15444418,1.8411744,-0.07922225,0.6559264,1.9780371,2.5806060,1.6749832,2.04978755,⋯,-1.0148940,-0.59314323,0.65505136,-0.5693505,-0.48892918,0.2077396965,-0.8016505,-0.28439422,-1.6336384,-0.65163455
RP11-34P13.15,-0.21703290,-1.2491072,-0.71067536,-0.5458638,-0.88156172,-0.4026354,-0.5965898,0.2922380,-0.4949213,0.05590706,⋯,-0.1934056,0.33783682,1.59084836,0.4045636,0.05473854,1.0872298726,0.5077824,-0.19027836,-0.9777920,0.24859783
RP11-34P13.16,0.05000362,-1.2529810,-1.26148519,-0.7514709,-0.77242737,-0.3766090,-0.9801061,0.4096420,-0.3255995,-0.06358077,⋯,0.1692925,0.70250273,1.43810645,0.6769784,0.17310765,1.2674193527,0.8571877,-0.10447332,-1.0448862,0.46041845
RP11-34P13.18,0.80964288,-0.4648762,0.60984294,0.6852051,-1.14329360,-0.5356296,1.4716736,1.0668942,0.1565502,0.90028575,⋯,-1.3100707,-0.67305864,-0.05994370,-1.0821739,-0.70715756,0.0894613755,-1.3990557,-0.65677160,-1.7141016,-0.37942569
AP006222.2,0.60340589,-0.6590192,-0.22788724,-0.3186479,-0.75919490,-0.8022825,-0.8073561,-0.9128508,-0.3943123,-0.90819239,⋯,0.1568485,1.49184552,1.31865074,-0.1807585,0.13219755,2.1816052692,-0.3754781,-0.03196266,-0.2097524,3.06398139
MTND1P23,-0.28701222,0.4584699,-0.04655404,-0.6834497,-0.28493094,0.5112355,-0.4015529,-0.5931863,0.1299671,-0.25562386,⋯,1.0400459,-0.03813723,-0.04317319,-0.1047992,0.06352471,0.0003105984,-0.2519768,-0.01557983,0.3048702,-0.02531932


In [20]:
saveRDS(df_gtex_fbm_filt, file = file.path(output_data_dir, "df_gtex_fbm_filt.rds"))
write.csv(df_gtex_fbm_filt, file = file.path(output_data_dir, "df_gtex_fbm_filt.csv"))

## SVD computation

In [21]:
if (!file.exists(file.path(output_data_dir, "gtex_svdRes.rds"))) {

  g_fb <- nrow(gtex_fbm_filt)
  samples_fb <- ncol(gtex_fbm_filt)
  SVD_K_gtex <- min(g_fb, samples_fb) - 1

  message("Using SVD K = ", SVD_K_gtex)

  if (N_CORES > 1) {
    # disable BLAS parallelization when parallelizing
    options(bigstatsr.check.parallel.blas = FALSE)
    blas_nproc <- getOption("default.nproc.blas")
    options(default.nproc.blas = NULL)
  }

  gtex_svdRes <- big_randomSVD(
    gtex_fbm_filt,
    k = SVD_K_gtex,
    ncores = N_CORES
  )

  saveRDS(gtex_svdRes, file = file.path(output_data_dir, "gtex_svdRes.rds"))

  if (N_CORES > 1) {
    # restore previous state
    options(bigstatsr.check.parallel.blas = TRUE)
    options(default.nproc.blas = blas_nproc)
  }

} else {
  message("gtex_svdRes already exists, skipping SVD computation.")
}

gtex_svdRes already exists, skipping SVD computation.



## Estimate K for CLAMP

In [22]:
gtex_fbm_filt <- readRDS(file.path(output_data_dir, "gtex_fbm_filt.rds"))
gtex_svdRes <- readRDS(file.path(output_data_dir, "gtex_svdRes.rds"))

In [23]:
CLAMP_K_gtex <- num.pc(list(d = gtex_svdRes$d)) * 2
message("Inferred CLAMP K = ", CLAMP_K_gtex)

Inferred CLAMP K = 412



In [24]:
saveRDS(CLAMP_K_gtex, file = file.path(output_data_dir, "CLAMP_K_gtex.rds"))

## CLAMPbase

In [24]:
gtex_baseRes <- CLAMPbase(
  Y      = gtex_fbm_filt,
  svdres = gtex_svdRes,
  trace  = TRUE,
  clamp_k = CLAMP_K_gtex
)

****

CLAMP k is set to 412

L1 is set to 45.2778145717356

L2 is set to 135.833443715207

Progress 1 / 200 | Bdiff=0.222227, minCor=0.616286

Progress 2 / 200 | Bdiff=0.030933, minCor=0.915387

Progress 3 / 200 | Bdiff=0.014314, minCor=0.958797

Progress 4 / 200 | Bdiff=0.009708, minCor=0.976381

Progress 5 / 200 | Bdiff=0.007364, minCor=0.983586

Progress 6 / 200 | Bdiff=0.005958, minCor=0.987160

Progress 7 / 200 | Bdiff=0.005016, minCor=0.988719

Progress 8 / 200 | Bdiff=0.004328, minCor=0.989041

Progress 9 / 200 | Bdiff=0.003798, minCor=0.989755

Progress 10 / 200 | Bdiff=0.003379, minCor=0.990697

Progress 11 / 200 | Bdiff=0.003044, minCor=0.992363

Progress 12 / 200 | Bdiff=0.002777, minCor=0.994038

Progress 13 / 200 | Bdiff=0.002561, minCor=0.994884

Progress 14 / 200 | Bdiff=0.002384, minCor=0.995539

Progress 15 / 200 | Bdiff=0.002234, minCor=0.995313

Progress 16 / 200 | Bdiff=0.002102, minCor=0.995279

Progress 17 / 200 | Bdiff=0.001984, minCor=0.995572

Progress 18 / 200

In [25]:
gtex_baseRes$Z <- data.frame(gtex_baseRes$Z)
rownames(gtex_baseRes$Z) <- gtex_genes
head(gtex_baseRes$Z)

gtex_baseRes$B <- data.frame(gtex_baseRes$B)
colnames(gtex_baseRes$B) <- samples
head(gtex_baseRes$B)

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,⋯,LV403,LV404,LV405,LV406,LV407,LV408,LV409,LV410,LV411,LV412
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
WASH7P,0,0.0000000,0.2238024,0,0.0000000,0.0000000,0,0,0,0,⋯,0,0.0000000,0.1889670,0.1215804,0,0.0000000,0,0,0,0
RP11-34P13.15,0,0.0000000,0.5096340,0,0.2509535,0.0000000,0,0,0,0,⋯,0,0.1674211,0.0000000,0.0000000,0,0.2696572,0,0,0,0
RP11-34P13.16,0,0.0000000,0.4357990,0,0.2101712,0.0000000,0,0,0,0,⋯,0,0.1554509,0.0000000,0.0000000,0,0.2536335,0,0,0,0
RP11-34P13.18,0,0.0000000,0.0000000,0,0.0000000,0.0000000,0,0,0,0,⋯,0,0.0000000,0.0000000,0.0000000,0,0.0000000,0,0,0,0
AP006222.2,0,0.3627945,0.5001041,0,0.0000000,0.2539736,0,0,0,0,⋯,0,0.7860915,0.0000000,0.2167742,0,0.0000000,0,0,0,0
MTND1P23,0,0.0000000,0.0000000,0,0.0000000,0.0000000,0,0,0,0,⋯,0,0.0000000,0.1661677,0.0000000,0,0.1903921,0,0,0,0


,GTEX-1117F-0226-SM-5GZZ7,GTEX-1117F-0426-SM-5EGHI,GTEX-1117F-0526-SM-5EGHJ,GTEX-1117F-0626-SM-5N9CS,GTEX-1117F-0726-SM-5GIEN,GTEX-1117F-1326-SM-5EGHH,GTEX-1117F-2426-SM-5EGGH,GTEX-1117F-2526-SM-5GZY6,GTEX-1117F-2826-SM-5GZXL,GTEX-1117F-2926-SM-5GZYI,⋯,GTEX-ZZPU-1126-SM-5N9CW,GTEX-ZZPU-1226-SM-5N9CK,GTEX-ZZPU-1326-SM-5GZWS,GTEX-ZZPU-1426-SM-5GZZ6,GTEX-ZZPU-1826-SM-5E43L,GTEX-ZZPU-2126-SM-5EGIU,GTEX-ZZPU-2226-SM-5EGIV,GTEX-ZZPU-2426-SM-5E44I,GTEX-ZZPU-2626-SM-5E45Y,GTEX-ZZPU-2726-SM-5NQ8O
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
LV1,0.40227829,-0.83067229,0.13767685,0.33399372,-0.02835619,0.3332145,0.447306650,0.338362604,0.331359274,0.28349952,⋯,-0.1829688,0.161323757,0.23365724,-0.06452863,0.03186911,0.41299403,0.18769557,0.08285806,-0.95743466,0.23072471
LV2,-0.21105905,-0.39058734,-0.25020511,-0.15217045,-0.22405753,-0.2509356,-0.068004330,-0.218217029,-0.276129991,-0.24454012,⋯,-0.2052221,-0.105190032,-0.17922709,-0.24780284,-0.20711392,-0.06367100,-0.35472026,-0.21735892,-0.39575569,-0.34217186
LV3,0.06087593,-0.30058551,-0.03465706,0.04175187,-0.19368120,-0.1413011,0.042950582,0.006486519,-0.006719105,-0.06047388,⋯,-0.1202926,-0.031512945,0.05337323,-0.06466048,-0.05868799,-0.13505090,-0.02195829,-0.03738955,-0.29465899,0.04364233
LV4,0.81015806,-0.13555167,0.33356367,0.46935608,0.30412657,0.3997392,0.554775946,-0.601353184,0.372632806,-0.09721415,⋯,0.2659180,0.004568919,0.04125612,-0.14726854,0.21557921,0.51003780,-0.99862885,0.24476964,-0.22761480,0.41206306
LV5,-0.07189147,-0.21222310,-0.19918345,-0.18590604,-0.24453001,-0.2324514,-0.208229526,1.902841596,-0.091063568,0.85135279,⋯,-0.1964528,-0.137560258,-0.03037793,-0.03205531,-0.24260611,-0.18757958,1.80884692,-0.27164005,-0.25716598,-0.21957204
LV6,0.07853258,0.08421465,0.04270353,-0.16429505,-0.10721337,-0.1239587,0.003039069,-0.091955043,-0.029737893,-0.06461595,⋯,-0.1120285,-0.163521751,-0.07320061,-0.07368514,-0.08754915,-0.01867507,-0.08342165,-0.17803233,0.02501471,-0.10536676


In [26]:
saveRDS(gtex_baseRes, file = file.path(output_data_dir, "CLAMPbase.rds"))

In [27]:
model_dir <- file.path(output_data_dir, "CLAMPbase")
dir.create(model_dir, showWarnings = FALSE, recursive = TRUE)

B <- gtex_baseRes$B
write.csv(B, file.path(model_dir, "B.csv"))

Z <- gtex_baseRes$Z
write.csv(Z, file.path(model_dir, "Z.csv"))

## Prepare pathway priors

In [25]:
gtex_gmtList <- list(
  KEGG = getGMT("https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=KEGG_2021_Human"),
  BP = getGMT("https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=GO_Biological_Process_2025"),
  GTEx_Tissues = getGMT("https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=GTEx_Tissues_V8_2023")
)

# prefix each gene‐set name with its library to guarantee uniqueness
for(lib in names(gtex_gmtList)) {
  names(gtex_gmtList[[lib]]) <- paste0(lib, "_", names(gtex_gmtList[[lib]]))
}

gtex_pathMat <- gmtListToSparseMat(gtex_gmtList)
gtex_matched <- getMatchedPathwayMat(gtex_pathMat, gtex_genes)
gtex_chatObj <- getChat(gtex_matched)

Auto-detected name: KEGG_2021_Human

Using cached file for KEGG_2021_Human

Auto-detected name: GO_Biological_Process_2025

Using cached file for GO_Biological_Process_2025

Auto-detected name: GTEx_Tissues_V8_2023

Using cached file for GTEx_Tissues_V8_2023

There are 13687 genes in the intersection between data and prior

Removing 2030 pathways

Inverting...

done



## CLAMPfull

In [29]:
gtex_fullRes <- CLAMPfull(
    Y = gtex_fbm_filt,
    svdres = gtex_svdRes,
    priorMat = gtex_matched,
    clamp.base.result = gtex_baseRes,
    use_cpp = TRUE,
    trace = TRUE,
    clamp_k = CLAMP_K_gtex
  )

** CLAMPfull **

using provided CLAMPbase result

CLAMP k is set to 412

L1=45.2778145717356; L2=135.833443715207

Progress 1 / 30 | Bdiff=0.000454

Progress 2 / 30 | Bdiff=0.038152

Progress 3 / 30 | Bdiff=0.015330

Estimated total runtime: ~7.0 min

Progress 4 / 30 | Bdiff=0.012882

Progress 5 / 30 | Bdiff=0.009090

Progress 6 / 30 | Bdiff=0.006588

Progress 7 / 30 | Bdiff=0.005506

Progress 8 / 30 | Bdiff=0.005159

Progress 9 / 30 | Bdiff=0.005130

Progress 10 / 30 | Bdiff=0.004874

Progress 11 / 30 | Bdiff=0.004892

Progress 12 / 30 | Bdiff=0.004566

Progress 13 / 30 | Bdiff=0.003983

Progress 14 / 30 | Bdiff=0.003996

Progress 15 / 30 | Bdiff=0.003731

Progress 16 / 30 | Bdiff=0.003469

Progress 17 / 30 | Bdiff=0.003616

Progress 18 / 30 | Bdiff=0.003057

Progress 19 / 30 | Bdiff=0.003160

Progress 20 / 30 | Bdiff=0.002978

Progress 21 / 30 | Bdiff=0.002880

Progress 22 / 30 | Bdiff=0.003017

Progress 23 / 30 | Bdiff=0.003014

Progress 24 / 30 | Bdiff=0.002967

Progress 25 / 30 | 

In [30]:
gtex_baseRes$Z <- data.frame(gtex_baseRes$Z)
rownames(gtex_baseRes$Z) <- gtex_genes
head(gtex_baseRes$Z)

gtex_baseRes$B <- data.frame(gtex_baseRes$B)
colnames(gtex_baseRes$B) <- samples
head(gtex_baseRes$B)

gtex_fullRes$summary <- gtex_fullRes$summary %>%
    dplyr::rename(LV = LV_index)  %>% 
    dplyr::mutate(LV = paste0('LV', LV))

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,⋯,LV403,LV404,LV405,LV406,LV407,LV408,LV409,LV410,LV411,LV412
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
WASH7P,0,0.0000000,0.2238024,0,0.0000000,0.0000000,0,0,0,0,⋯,0,0.0000000,0.1889670,0.1215804,0,0.0000000,0,0,0,0
RP11-34P13.15,0,0.0000000,0.5096340,0,0.2509535,0.0000000,0,0,0,0,⋯,0,0.1674211,0.0000000,0.0000000,0,0.2696572,0,0,0,0
RP11-34P13.16,0,0.0000000,0.4357990,0,0.2101712,0.0000000,0,0,0,0,⋯,0,0.1554509,0.0000000,0.0000000,0,0.2536335,0,0,0,0
RP11-34P13.18,0,0.0000000,0.0000000,0,0.0000000,0.0000000,0,0,0,0,⋯,0,0.0000000,0.0000000,0.0000000,0,0.0000000,0,0,0,0
AP006222.2,0,0.3627945,0.5001041,0,0.0000000,0.2539736,0,0,0,0,⋯,0,0.7860915,0.0000000,0.2167742,0,0.0000000,0,0,0,0
MTND1P23,0,0.0000000,0.0000000,0,0.0000000,0.0000000,0,0,0,0,⋯,0,0.0000000,0.1661677,0.0000000,0,0.1903921,0,0,0,0


,GTEX-1117F-0226-SM-5GZZ7,GTEX-1117F-0426-SM-5EGHI,GTEX-1117F-0526-SM-5EGHJ,GTEX-1117F-0626-SM-5N9CS,GTEX-1117F-0726-SM-5GIEN,GTEX-1117F-1326-SM-5EGHH,GTEX-1117F-2426-SM-5EGGH,GTEX-1117F-2526-SM-5GZY6,GTEX-1117F-2826-SM-5GZXL,GTEX-1117F-2926-SM-5GZYI,⋯,GTEX-ZZPU-1126-SM-5N9CW,GTEX-ZZPU-1226-SM-5N9CK,GTEX-ZZPU-1326-SM-5GZWS,GTEX-ZZPU-1426-SM-5GZZ6,GTEX-ZZPU-1826-SM-5E43L,GTEX-ZZPU-2126-SM-5EGIU,GTEX-ZZPU-2226-SM-5EGIV,GTEX-ZZPU-2426-SM-5E44I,GTEX-ZZPU-2626-SM-5E45Y,GTEX-ZZPU-2726-SM-5NQ8O
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
LV1,0.40227829,-0.83067229,0.13767685,0.33399372,-0.02835619,0.3332145,0.447306650,0.338362604,0.331359274,0.28349952,⋯,-0.1829688,0.161323757,0.23365724,-0.06452863,0.03186911,0.41299403,0.18769557,0.08285806,-0.95743466,0.23072471
LV2,-0.21105905,-0.39058734,-0.25020511,-0.15217045,-0.22405753,-0.2509356,-0.068004330,-0.218217029,-0.276129991,-0.24454012,⋯,-0.2052221,-0.105190032,-0.17922709,-0.24780284,-0.20711392,-0.06367100,-0.35472026,-0.21735892,-0.39575569,-0.34217186
LV3,0.06087593,-0.30058551,-0.03465706,0.04175187,-0.19368120,-0.1413011,0.042950582,0.006486519,-0.006719105,-0.06047388,⋯,-0.1202926,-0.031512945,0.05337323,-0.06466048,-0.05868799,-0.13505090,-0.02195829,-0.03738955,-0.29465899,0.04364233
LV4,0.81015806,-0.13555167,0.33356367,0.46935608,0.30412657,0.3997392,0.554775946,-0.601353184,0.372632806,-0.09721415,⋯,0.2659180,0.004568919,0.04125612,-0.14726854,0.21557921,0.51003780,-0.99862885,0.24476964,-0.22761480,0.41206306
LV5,-0.07189147,-0.21222310,-0.19918345,-0.18590604,-0.24453001,-0.2324514,-0.208229526,1.902841596,-0.091063568,0.85135279,⋯,-0.1964528,-0.137560258,-0.03037793,-0.03205531,-0.24260611,-0.18757958,1.80884692,-0.27164005,-0.25716598,-0.21957204
LV6,0.07853258,0.08421465,0.04270353,-0.16429505,-0.10721337,-0.1239587,0.003039069,-0.091955043,-0.029737893,-0.06461595,⋯,-0.1120285,-0.163521751,-0.07320061,-0.07368514,-0.08754915,-0.01867507,-0.08342165,-0.17803233,0.02501471,-0.10536676


In [31]:
saveRDS(gtex_fullRes, file = file.path(output_data_dir, "CLAMPfull.rds"))

In [32]:
model_dir <- file.path(output_data_dir, "CLAMPfull")
dir.create(model_dir, showWarnings = FALSE, recursive = TRUE)

B <- gtex_fullRes$B
write.csv(B, file.path(model_dir, "B.csv"))

Z <- gtex_fullRes$Z
write.csv(Z, file.path(model_dir, "Z.csv"))

summary <- gtex_fullRes$summary
write.csv(summary, file.path(model_dir, "summary.csv"))

In [33]:
# Create a timestamp to track the end of the analysis
end_time <- Sys.time()
elapsed_time <- end_time - start_time
cat("GTEx CLAMP analysis completed at:", format(end_time), "\n")
cat("Total elapsed time:", format(elapsed_time), "\n")

GTEx CLAMP analysis completed at: 2025-11-18 11:21:04 
Total elapsed time: 26.8807 mins 


# PLIER

Run PLIER with the same inputs

In [26]:
gtex_plier = PLIER::PLIER(
    gtex_fbm_filt[], 
    as.matrix(gtex_matched), 
    svdres = gtex_svdRes, 
    Chat = as.matrix(gtex_chatObj), 
    doCrossval = TRUE, 
    k = CLAMP_K_gtex
  )

Removing 0 pathways with too few genes



[1] 135.8334
[1] "L2 is set to 135.833443715207"
[1] "L1 is set to 67.9167218576034"


errorY (SVD based:best possible) = 0.3614

New L3 is 0.000335462627902512

New L3 is 0.000179560205425833

New L3 is 0.000158461325115751

New L3 is 0.000179560205425833

New L3 is 0.000158461325115751

New L3 is 0.000158461325115751

New L3 is 0.000179560205425833

New L3 is 0.000179560205425833

New L3 is 0.000179560205425833

New L3 is 0.000179560205425833

New L3 is 0.000179560205425833

New L3 is 0.000203468369010644

New L3 is 0.000203468369010644

Bdiff is not decreasing

Bdiff is not decreasing

Bdiff is not decreasing

Bdiff is not decreasing

Bdiff is not decreasing

Bdiff is not decreasing

converged at  iteration 270 Bdiff is not decreasing

There are 231  LVs with AUC>0.70



In [27]:
colnames(gtex_plier$Z) <- paste0('LV', seq_len(ncol(gtex_plier$Z)))

In [28]:
head(gtex_plier$Z)
dim(gtex_plier$Z)

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,⋯,LV403,LV404,LV405,LV406,LV407,LV408,LV409,LV410,LV411,LV412
WASH7P,0.07504047,0.00000000,0.000000000,0.00000000,0.04401260,0.002607857,0,0.00000000,0.03542812,0.00000000,⋯,0.000000000,0,0.078962190,0.0000000000,0.01982962,0.00000000,0.00000000,0.00000000,0.0000000,0.00000000
RP11-34P13.15,0.00000000,0.00000000,0.058503974,0.79492778,0.03502098,0.279668501,0,0.06675880,0.00000000,0.00000000,⋯,0.002741689,0,0.000000000,0.0504606776,0.00000000,0.00000000,0.00000000,0.03376556,0.0000000,0.02758407
RP11-34P13.16,0.14621480,0.00000000,0.053352372,0.84101647,0.03331726,0.269629187,0,0.08743634,0.00000000,0.00000000,⋯,0.000000000,0,0.000000000,0.0560132490,0.00000000,0.00000000,0.00000000,0.00000000,0.0000000,0.01643925
RP11-34P13.18,0.18986370,0.00000000,0.000000000,0.00000000,0.02569342,0.165715339,0,0.00000000,0.03397189,0.00000000,⋯,0.000000000,0,0.047208804,0.0243302725,0.00000000,0.05661247,0.00000000,0.00000000,0.1011321,0.00283420
AP006222.2,0.02956592,0.00000000,0.008740044,0.09701475,0.05976653,0.018426393,0,0.00000000,0.00000000,0.00000000,⋯,0.000000000,0,0.000000000,0.0005711404,0.00000000,0.00000000,0.00000000,0.11215684,0.0736887,0.05973406
MTND1P23,0.18452081,0.05828998,0.000000000,0.05636594,0.04971311,0.022160873,0,0.00000000,0.00000000,0.03867309,⋯,0.000000000,0,0.008344327,0.0598803344,0.00000000,0.17578418,0.04881264,0.05721422,0.0000000,0.00000000


[1] 21613   412

In [29]:
gtex_plier$summary <- gtex_plier$summary %>%
    dplyr::rename(LV = `LV index`)  %>% 
    dplyr::mutate(LV = paste0('LV', LV))

In [30]:
head(gtex_plier$summary)

,pathway,LV,AUC,p-value,FDR
,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,KEGG_Central carbon metabolism in cancer,LV1,0.7495416,1.348543e-03,2.758274e-03
2,KEGG_Glucagon signaling pathway,LV1,0.6850573,3.219539e-03,6.020819e-03
3,KEGG_Hepatocellular carcinoma,LV1,0.6797535,2.586467e-04,6.170776e-04
4,KEGG_Huntington disease,LV1,0.6990680,4.149301e-07,1.597248e-06
5,KEGG_Lysosome,LV1,0.6789606,1.175755e-03,2.444257e-03
6,KEGG_MicroRNAs in cancer,LV1,0.6080717,1.843455e-02,2.856891e-02


In [31]:
head(gtex_plier$B)
dim(gtex_plier$B)

"1,BP_mRNA Processing (GO:0006397)",0.4132567949,0.486759327,0.663813527,0.03690419,-0.33039549,0.06203486,0.17256557,0.38518837,0.153326446,0.54599180,⋯,-0.064012898,0.104904005,0.15994639,-0.022617591,0.51445116,0.414828975,0.64351090,0.61545800,0.540151920,0.25154228
"2,GTEx_Tissues_Brain - Cerebellar Hemisphere Male 70-79 Up",-0.0003651402,0.026278812,-0.006134067,-0.02715862,0.04729688,0.02568044,-0.04994434,-0.05689057,-0.010196194,0.02642679,⋯,-0.009448991,-0.005044218,-0.06498598,-0.008957803,-0.06211177,-0.035863571,-0.02482984,-0.05699719,0.007765338,-0.02250420
"3,GTEx_Tissues_Esophagus - Mucosa Male 20-29 Up",-0.0260914765,0.195752718,0.109714786,-0.02914759,0.02985596,-0.01235609,-0.06265101,0.33804811,-0.064094753,0.03263120,⋯,0.011438632,-0.051664338,-0.06741194,-0.084221493,0.03122792,0.001765275,0.56750539,0.03761025,0.005079531,-0.03112858
"4,GTEx_Tissues_Whole Blood Female 40-49 Up",-0.1097829857,0.005895992,-0.084289251,-0.14711402,0.03346850,-0.10708743,-0.08474846,-0.15071471,-0.068560044,-0.13618291,⋯,0.036846354,-0.171708071,-0.08837954,-0.078822658,-0.09553386,-0.168626183,-0.09504550,-0.02527299,0.001797163,-0.01129066
"5,GTEx_Tissues_Muscle - Skeletal Female 30-39 Up",0.0206195118,1.105339493,0.026924165,-0.05750232,0.02695948,-0.06832356,-0.07898940,-0.16185025,0.006030565,-0.07548849,⋯,0.089134340,-0.092464552,-0.13993025,-0.107671551,0.01376060,-0.266175645,-0.11054268,-0.03040460,1.899969030,0.10479688
"6,GTEx_Tissues_Testis Male 60-69 Up",-0.0060083814,0.008351103,-0.031526140,-0.06424883,-0.07858955,-0.09137689,-0.04069983,-0.12065652,-0.040161752,-0.10516739,⋯,-0.093828504,-0.113598445,-0.03352100,-0.111505441,-0.04595042,-0.153324774,-0.16227628,-0.05175557,-0.070849925,-0.03938802


[1]   412 17382

In [32]:
colnames(gtex_plier$B) <- samples

In [33]:
saveRDS(gtex_plier, file = file.path(output_data_dir, "PLIER.rds"))

In [34]:
model_dir <- file.path(output_data_dir, "PLIER")
dir.create(model_dir, showWarnings = FALSE, recursive = TRUE)

B <- gtex_plier$B
write.csv(B, file.path(model_dir, "B.csv"))

Z <- gtex_plier$Z
write.csv(Z, file.path(model_dir, "Z.csv"))

summary <- gtex_plier$summary
write.csv(summary, file.path(model_dir, "summary.csv"))